## df_shuffle.csv


| Hour of Day | Index |
| ----------- | ----- |
| 00:00       | 0     |
| 01:00       | 1     |
| ...         | ...   |
| 23:00       | 23    |

In [4]:
sc

In [11]:
import pyproj
import csv
import shapely.geometry as geom
import fiona
import fiona.crs
import shapely
import rtree
import geopandas as gpd
import numpy as np
import operator
# just for display, not for calculation
import pandas as pd
import datetime

In [12]:
# df = pd.read_csv('./df_shuffle.csv')
# df.head(10)

# def wkd(time):
#     return datetime.datetime.strptime(time, '%Y-%m-%d %H:%M:%S').weekday()

# wkd(df.tpep_pickup_datetime[0])
# pd.to_datetime(df.tpep_pickup_datetime).map(lambda x: x.weekday())

# Function

In [13]:
def countLine(partID, records):
    import pyproj
    import csv
    import shapely.geometry as geom
    import fiona
    import fiona.crs
    import shapely
    import rtree
    import geopandas as gpd
    import numpy as np
    import operator
    import pandas as pd
    import datetime
    
    shapefile = '/Users/chunqingxu/Documents/Spring2017/BDMA/Project/why_yellow_taxi/Buffer/entr_buffer_100_feet_epsg4269_nad83/entr_buffer_100_feet_epsg4269_nad83.shp'
    entr_buf = gpd.read_file(shapefile)
    entr_buf = entr_buf.to_crs(fiona.crs.from_epsg(2263))
    
    routes = ['Route_' + str(n) for n in range(1, 12)]
    entr2line = []
    for i in xrange(len(entr_buf)):
        lines = []
        for line in list(entr_buf.loc[:,routes].ix[i].dropna().values):
            try:
                line = str(int(line))
            except ValueError:
                pass
            lines.append(line)
        entr2line.append(lines)
    entr_buf['entr2line'] = entr2line
    
    index = rtree.Rtree()
    for idx, geometry in enumerate(entr_buf.geometry):
        index.insert(idx, geometry.bounds)
    

    entr_pair = {}
    pick_entr = {}
    drop_entr = {}
    entr_lines = {}
    
    proj = pyproj.Proj(init='epsg:2263', preserve_units=True)
    
    if partID==0:
        records.next()
    reader = csv.reader(records)
    for row in reader:
        if ((float(row[5])!=0) and float(row[9]!=0)):
            if row[1]:
                h = datetime.datetime.strptime(row[1], '%Y-%m-%d %H:%M:%S').hour
            else:
                h = None
    
            p = geom.Point(proj(float(row[5]), float(row[6])))
            d = geom.Point(proj(float(row[9]), float(row[10])))
            p_potential = index.intersection((p.x,p.y,p.x,p.y))
            d_potential = index.intersection((d.x,d.y,d.x,d.y))
            p_match = None # The first one match, should be the closest one? No!
            d_match = None
            
            for p_idx in p_potential:
                if entr_buf.geometry[p_idx].contains(p):
                    p_match = p_idx # print 'p',p_idx
                    p_lines = set(entr_buf.entr2line[p_idx])
                    break
            pick_entr[p_match] = pick_entr.get(p_match, 0)+1
            
            for d_idx in d_potential:
                if entr_buf.geometry[d_idx].contains(d):
                    d_match = d_idx # print 'd',d_idx
                    d_lines = set(entr_buf.entr2line[d_idx])
                    break
            drop_entr[d_match] = drop_entr.get(d_match, 0)+1
            
            if ((p_match and d_match) and (p_match != d_match)):
                dirct_lines = tuple(p_lines.intersection(d_lines))
                dirct_lines_h = (dirct_lines, h)
                if dirct_lines:
                    entr_lines[dirct_lines_h] = entr_lines.get(dirct_lines_h, 0)+1
                if p_match > d_match:
                    pair = (d_match, p_match)
                else:
                    pair = (p_match, d_match)
                entr_pair[pair] = entr_pair.get(pair, 0)+1
                
    return entr_lines.items()

In [14]:
def mapper(record):
    for key in record[0][0]:
        yield (key, record[0][1]), record[1]
        
rdd = sc.textFile('/Users/chunqingxu/Documents/Spring2017/BDMA/Project/why_yellow_taxi/Data/df_shuffle.csv')
counts = rdd.mapPartitionsWithIndex(countLine).flatMap(mapper).reduceByKey(lambda x,y: x+y).collect()  

In [15]:
for i in range(len(counts)):
    if counts[i][0][0] == '6':
        print 'Hours of day:{} ; Counts:{}'.format(counts[i][0][1], counts[i][1])

Hours of day:13 ; Counts:2
Hours of day:11 ; Counts:2
Hours of day:21 ; Counts:1
Hours of day:14 ; Counts:1
Hours of day:16 ; Counts:1


In [16]:
sorted(counts, key=lambda x: x[1], reverse=True)

[(('6', 13), 2),
 ((u'E', 10), 2),
 ((u'C', 10), 2),
 (('1', 14), 2),
 (('6', 11), 2),
 ((u'F', 14), 2),
 (('1', 13), 2),
 (('1', 17), 2),
 ((u'L', 20), 2),
 (('2', 9), 1),
 ((u'M', 14), 1),
 (('3', 14), 1),
 ((u'C', 20), 1),
 ((u'C', 6), 1),
 (('7', 8), 1),
 (('1', 20), 1),
 ((u'E', 20), 1),
 (('5', 16), 1),
 (('1', 6), 1),
 (('1', 16), 1),
 ((u'E', 6), 1),
 ((u'N', 21), 1),
 (('1', 2), 1),
 ((u'A', 6), 1),
 ((u'A', 2), 1),
 ((u'M', 8), 1),
 (('7', 20), 1),
 ((u'F', 21), 1),
 ((u'R', 19), 1),
 ((u'E', 8), 1),
 (('2', 13), 1),
 ((u'F', 7), 1),
 ((u'R', 15), 1),
 (('6', 21), 1),
 ((u'A', 10), 1),
 ((u'N', 19), 1),
 (('3', 20), 1),
 (('1', 12), 1),
 (('1', 22), 1),
 (('2', 17), 1),
 ((u'Q', 18), 1),
 (('3', 6), 1),
 ((u'C', 2), 1),
 ((u'Q', 16), 1),
 ((u'L', 11), 1),
 ((u'N', 15), 1),
 ((u'R', 21), 1),
 (('1', 5), 1),
 ((u'B', 2), 1),
 (('3', 17), 1),
 ((u'Q', 19), 1),
 ((u'C', 17), 1),
 ((u'R', 18), 1),
 ((u'R', 16), 1),
 ((u'C', 21), 1),
 ((u'A', 13), 1),
 (('6', 14), 1),
 (('1', 21), 